In [6]:
import numpy as np 
import matplotlib.pyplot as plt 
import sympy as smp 
from sympy.functions import conjugate
from scipy.integrate import nquad

# Question

An electron is initially prepared in quantum mechanical state with zero avg momentum and a wavefunction 

$$ \psi(x,t=0)\ \propto e^{-\frac{x^2}{2a^2}} $$

The potential everywhere is zero. A detector is positioned at distance x = L from the electron's initial position.

1. After approximately how long, the electron likely to have passed the detector?
2. Find the probability of observing the electron at x = L as a function of time.

# Part 1

We can obtain an estimate by computing $\braket{p^2}$ from the wavefunction and then using the classical estimation p = mv = mL/t where t is the time to get to the detector. This gives

$$ t = \frac{mL}{\sqrt{\braket{p^2}}} $$

Ofcourse, since the particle starts with zero avg momentum, it can only have at most a 50% chance of passing the detector, We just need to compute $\braket{p^2}$, given by

$$ \braket{p^2}\ =\ \int_{-\infty}^{\infty} \psi^*\hat{p^2}\psi dx = \int_{-\infty}^{\infty} \psi^*\Big(-\frac{\partial^2}{\partial x^2}\psi\Big)dx $$

First, lets define all the varaibles we need, making sure to specify all their properties :

In [7]:
x = smp.symbols('x', real=True)
a, L, m, k, kp, t = smp.symbols('a L m k k\' t', real=True, positive=True)

Now, lets define and normalize the Wavefunction :

In [8]:
exp = smp.exp(-x**2/ (2*a**2))
psi = exp/smp.sqrt(smp.integrate(exp**2, (x, -smp.oo, smp.oo)))

In [9]:
exp

exp(-x**2/(2*a**2))

In [10]:
psi

exp(-x**2/(2*a**2))/(pi**(1/4)*sqrt(a))

Now, Computing the value of $\braket{p^2}$

In [11]:
p2 = smp.integrate(psi*(-smp.diff(psi, (x,2))), (x, -smp.oo, smp.oo))
p2

1/(2*a**2)

Now, we compute the time t given the equation above:

In [12]:
T = m*L/smp.sqrt(p2)
T

sqrt(2)*L*a*m

This does not currently have units of time (has units $kg.m^2$). Since $\hbar$ which we chose 1 has units $kg.m^2$, we simply need to divide by $\hbar$ if we want to do numeric computation

# Part 2

Above was simply an approximation. The wavefunction changes in time according to the Schrodinger Equation, which in the position basis, is given by :

$$ i\frac{\partial}{\partial t}\psi(x,t)\ =\ H\psi(x,t) $$

where, H = $-\frac{1}{2m}\frac{\partial^2}{\partial x^2}$ is the Hamiltonian linear operator for a particle in a free space. the general solution to this equation is given by

$$ \psi(x,t)\ =\ e^{iHt}\psi(x,t=0) $$

where $e^A \equiv \sum_{n=0}^{\infty} \frac{A^n}{n!}.$ Since we know $\psi(x,t=0)$ and we know H, we essentially have the solution! it's just not in the convenient form. so if we can express $\psi(x,t=0$ in terms of the eigenstates of H as $\psi(x,0)\ =\ \int_{E} \psi_{E}(x)(E)dE_c,$ then the solution is

$$ \psi(x,t)\ = \ e^{iHt}\psi(x,t=0) $$
$$ =\ e^{iHt}\int_{E}\psi_{E}(x)(E)dE_c $$
$$ =\ \int_{E}\psi_{E}(x)e^{iEt}(E)dE_c $$

For this we need the eigenvalues and eigenstates of H. For this, we use H = $-\frac{1}{2m}\frac{\partial^2}{\partial x^2}$ and search for eigenstates

$$ H\psi = E\psi $$
$$ -\frac{1}{2m}\frac{\partial^2}{\partial x^2}\psi =\ E\psi $$
$$ \frac{\partial^2}{\partial x^2}\psi =\ -2mE\psi $$

Our characteristic length scale of the problem is a, so ,multiplying this eqn. by a and redefining x such that 

$$ \frac{\partial^2}{\partial x^2}\psi =\ -2ma^2.E\psi $$

Let $2ma^2E\ =\ k^2$, then, we have $\frac{\partial^2}{\partial x^2}\psi =\ -2ma^2.E\psi $ and the solution to this eqn. is given by, $\psi_{k}\ =\ e^{ikx}.$
$$ \psi(x,0)\ =\ \int_{E}\psi_{E}(x)(E)dE_c\ =\ \int_{-\infty}^{\infty}c(k).e^{ikx}dk $$

So, the c(k)'s are just the inverse Fourier transform of $\psi(x,0)$:
$$ c(k)\ =\ \frac{1}{2\pi}\int_{-\infty}^{\infty} \psi(x,0)e^{-ikx}dx $$

We have all the information to find these. FIrst lets redefine $\psi$ using x $\rightarrow$ x/a

In [13]:
psi = smp.exp(-x**2/2)
psi = psi/smp.sqrt(smp.integrate(psi**2, (x, -smp.oo, smp.oo)))

In [14]:
psi

exp(-x**2/2)/pi**(1/4)

Now, define $\psi_k(x)$ and get the c(k)'s

In [15]:
psi_k = smp.exp(smp.I*k*x)
c_k = 1/(2*smp.pi) * smp.integrate(psi*psi_k.conjugate(), (x, -smp.oo, smp.oo)).simplify()

In [16]:
psi_k

exp(I*k*x)

In [17]:
c_k

sqrt(2)*exp(-k**2/2)/(2*pi**(3/4))

Let's check to make sure we get our original wave function back by evaluating $\int_{-\infty}^{\infty}c(k).e^{ikx}dk$

In [18]:
smp.integrate(c_k*psi_k, (k, -smp.oo, smp.oo)).simplify()

exp(-x**2/2)/pi**(1/4)